# Cleaning the data + feature engineering

## Imports

In [ ]:
import os  #to get filepaths
import numpy as np
import pandas as pd

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 100)

In [ ]:
os.getcwd()

In [ ]:
info_file = '/Users/ryanrappa/Desktop/dsi/film-profit-prediction/csv_files/info_10k.csv'
credits_file = '/Users/ryanrappa/Desktop/dsi/film-profit-prediction/csv_files/credits_10k.csv'
releases_file = '/Users/ryanrappa/Desktop/dsi/film-profit-prediction/csv_files/releases_10k.csv'

In [ ]:
info_df = pd.read_csv(info_file)
credits_df = pd.read_csv(credits_file)
releases_df = pd.read_csv(releases_file)

## Cleaning

#### Because of the way data was collected, the last row contains the headers. Fixing that:

In [ ]:
info_df.columns = info_df.iloc[-1:, :].squeeze()
info_df = info_df.iloc[:-1, :]

In [ ]:
credits_df.columns = credits_df.iloc[-1:, :].squeeze()
credits_df = credits_df.iloc[:-1, :]

In [ ]:
releases_df.columns = releases_df.iloc[-1:, :].squeeze()
releases_df = releases_df.iloc[:-1, :]

In [ ]:
print(type(credits_df['cast'][2]))

In [ ]:
credits_df.head()

In [ ]:
lod = eval(credits_df['cast'][0])

In [ ]:
lod[1]

#### other experiment (tb deleted):

In [ ]:
# result = []
# for idx, row in credits_df.iterrows():
#     for dct in row['cast']:
#         dct['cast_id'] = row['cast_id']
#         dct['count'] = row['count']
#         result.append(dct)

In [ ]:
# print(type(credits_df['cast'][2]))
# credits_df.cast.apply(lambda s: list(literal_eval(s)))

#### json experiment (probably to be deleted):

In [ ]:
pd.io.json.json_normalize(test_df['cast_json'].apply(json.loads))

In [ ]:
json_credits = credits_df.iloc[:, 1:2].squeeze().str.replace('\'', '\"', )

In [ ]:
credits_df = pd.concat([credits_df, json_credits], axis=1)

In [ ]:
credits_df.columns = ['id', 'cast', 'crew', 'cast_json']